In [1]:
import polib
from openai import OpenAI
import time
from dotenv import load_dotenv
import os
from tqdm import tqdm
import glob

In [2]:
load_dotenv(override=True)
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [3]:
def get_completion(system_message, user_message, model="gpt-3.5-turbo"):
    client = OpenAI()
    
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message}
    ]
    completion = client.chat.completions.create(
        model=model,
        messages=messages
    )
    return completion.choices[0].message.content

def translate_text_to_japanese(text):
    system_message = "あなたはデジタルコレクションに関する専門家です。Omeka Sに関する開発者向けドキュメントを翻訳しています。以下のmdファイルの内容を日本語に翻訳して、翻訳結果をmdファイルの形式で返却してください。"

    response = get_completion(system_message, text, model="gpt-4-1106-preview") # , model="gpt-4"
    return response

def translate_md_file(input_po_path, output_po_path):
    """`.po`ファイルを読み込み、各メッセージを日本語に翻訳して新しい`.po`ファイルに保存します。"""

    with open(input_po_path, "r") as f:
        input_po = f.read()

        token_count = len(input_po.split())

        print(f"Translating {token_count} tokens in {input_po_path}...")

        translated_text = translate_text_to_japanese(input_po) # .replace("```markdown\n", "").replace("\n```", "")

        # Markdownのコードブロックの先頭と最後のバッククォートを除去する
        # 先頭のバッククォートを除去
        if translated_text.startswith("```markdown\n"):
            translated_text = translated_text[len("```markdown\n"):]

        # 最後のバッククォートを除去
        if translated_text.endswith("\n```"):
            translated_text = translated_text[:-len("\n```")]

        with open(output_po_path, "w") as f:
            f.write(translated_text)

In [4]:
files = glob.glob("../docs/**/*.md", recursive=True)

files.sort()

In [5]:
for file in tqdm(files):

    basename = os.path.basename(file)

    if "ja" in basename:
        continue

    output_path = os.path.join(os.path.dirname(file), basename.replace(".md", ".ja.md"))

    # `.po`ファイルの翻訳を実行
    translate_md_file(file, output_path)

../docs/accessibility.md 1 of 65
Translating 281 tokens in ../docs/accessibility.md...


KeyboardInterrupt: 